In [93]:
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
import matplotlib.pylab as plt
import hashlib
import time
import pickle
%matplotlib inline

In [94]:
train = pd.read_csv('data/train.csv')
valid = pd.read_csv('data/valid.csv')
test = pd.read_csv('data/testa.csv')
train
train['content'] = train['content'].apply(lambda x :' '.join(jieba.cut(x)))
valid['content'] = valid['content'].apply(lambda x :' '.join(jieba.cut(x)))
test['content'] = test['content'].apply(lambda x :' '.join(jieba.cut(x)))

In [95]:
train.columns.tolist()
label_list = train.columns.tolist()[2:]
data_dict = {}
for label in label_list:
    tmp_dict = {}
    tmp_dict['binary_train'] = train.loc[:,['id','content',label]]
    tmp_dict['binary_train'].loc[tmp_dict['binary_train'][label]!=-2,label] = 1
    tmp_dict['binary_train'].loc[tmp_dict['binary_train'][label]==-2,label] = 0
    tmp_dict['binary_valid'] = valid.loc[:,['id','content',label]]
    tmp_dict['binary_valid'].loc[tmp_dict['binary_valid'][label]!=-2,label] = 1
    tmp_dict['binary_valid'].loc[tmp_dict['binary_valid'][label]==-2,label] = 0
    tmp_dict['sentiment_train'] = train.loc[:,['id','content',label]]
    tmp_dict['sentiment_train']=tmp_dict['sentiment_train'][~tmp_dict['sentiment_train'][label].isin([-2])]
    tmp_dict['sentiment_valid'] = valid.loc[:,['id','content',label]]
    tmp_dict['sentiment_valid']=tmp_dict['sentiment_valid'][~tmp_dict['sentiment_valid'][label].isin([-2])]
    tmp_dict['true_valid'] = valid.loc[:,['id','content',label]]
    data_dict[label] = tmp_dict

In [96]:
train

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,""" 吼吼 吼 ， 萌死 人 的 棒棒糖 ， 中 了 大众 点评 的 霸王餐 ， 太 可爱 了...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,""" 第三次 参加 大众 点评 网 霸王餐 的 活动 。 这家 店 给 人 整体 感觉 一般 ...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,""" 4 人 同行 点 了 10 个 小吃 \n 榴莲 酥 榴莲 味道 不足 松软...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,""" 之前 评价 了 莫名其妙 被删 果断 继续 差评 ！ 换 了 菜单 价格 更...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,""" 出乎意料 地 惊艳 ， 椰子 鸡 清热 降火 ， 美容 养颜 ， 大大 满足 了 爱 吃...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2
5,5,""" 烤鸭 使用 的 蘸 酱 很 地道 ， 觉得 烤鸭 好不好 吃 在 很大 程度 上 都 取...",-2,-2,-2,-2,1,-2,-2,0,...,-2,-2,1,-2,0,1,-2,-2,1,-2
6,6,""" 在 好友 圈 很 红 的 原因 很 诡异 . 竟然 基本 都 是 我 外地 的 朋友 转...",-2,-2,-1,-2,-2,-2,-2,-1,...,-2,-2,-1,-2,-1,1,1,1,1,-2
7,7,""" 中午饭 点人 比较 多 需要 排队 ～ 2 点 左右 去 吃 的 刚好 可以 用 团购 ...",-2,-2,-2,0,1,-2,-2,-2,...,-2,-2,-2,-1,-1,0,-2,1,1,1
8,8,""" 跟 朋友 相约 25 号 看港 囧 ， 首映 当天 排片 率 超高 ， 为了 提前 订好...",-2,1,-2,-2,-2,-2,-2,1,...,1,-2,-2,-2,-2,1,-2,-2,1,1
9,9,""" 首先 感谢 大众 点评 再次 抽中 我 免费 品尝 火锅 ， 同时 也 感谢 古蜀 雒味...",-2,-2,1,-2,1,-2,-2,-2,...,1,1,1,1,0,-2,1,-2,0,1


In [97]:
data_dict['service_waiters_attitude']['binary_train']

,id,content,service_waiters_attitude
0,0,""" 吼吼 吼 ， 萌死 人 的 棒棒糖 ， 中 了 大众 点评 的 霸王餐 ， 太 可爱 了...",1
1,1,""" 第三次 参加 大众 点评 网 霸王餐 的 活动 。 这家 店 给 人 整体 感觉 一般 ...",0
2,2,""" 4 人 同行 点 了 10 个 小吃 \n 榴莲 酥 榴莲 味道 不足 松软...",1
3,3,""" 之前 评价 了 莫名其妙 被删 果断 继续 差评 ！ 换 了 菜单 价格 更...",0
4,4,""" 出乎意料 地 惊艳 ， 椰子 鸡 清热 降火 ， 美容 养颜 ， 大大 满足 了 爱 吃...",0
5,5,""" 烤鸭 使用 的 蘸 酱 很 地道 ， 觉得 烤鸭 好不好 吃 在 很大 程度 上 都 取...",1
6,6,""" 在 好友 圈 很 红 的 原因 很 诡异 . 竟然 基本 都 是 我 外地 的 朋友 转...",0
7,7,""" 中午饭 点人 比较 多 需要 排队 ～ 2 点 左右 去 吃 的 刚好 可以 用 团购 ...",1
8,8,""" 跟 朋友 相约 25 号 看港 囧 ， 首映 当天 排片 率 超高 ， 为了 提前 订好...",0
9,9,""" 首先 感谢 大众 点评 再次 抽中 我 免费 品尝 火锅 ， 同时 也 感谢 古蜀 雒味...",1


In [98]:
data_dict['service_waiters_attitude']['sentiment_train']

,id,content,service_waiters_attitude
0,0,""" 吼吼 吼 ， 萌死 人 的 棒棒糖 ， 中 了 大众 点评 的 霸王餐 ， 太 可爱 了...",1
2,2,""" 4 人 同行 点 了 10 个 小吃 \n 榴莲 酥 榴莲 味道 不足 松软...",0
5,5,""" 烤鸭 使用 的 蘸 酱 很 地道 ， 觉得 烤鸭 好不好 吃 在 很大 程度 上 都 取...",1
7,7,""" 中午饭 点人 比较 多 需要 排队 ～ 2 点 左右 去 吃 的 刚好 可以 用 团购 ...",1
9,9,""" 首先 感谢 大众 点评 再次 抽中 我 免费 品尝 火锅 ， 同时 也 感谢 古蜀 雒味...",1
10,10,""" 这次 必须 给 全 五分 ！ 因为 太 完美 了 ！ 整个 就餐 体验 相当 棒 ！ \...",1
13,13,""" 很棒 的 一次 自助 体验 ～ 之前 就团 好 的 券 一直 延期 就 一直 推 ...",1
14,14,""" 在 星和园 后门 的 临街 铺面 ， 3 家 粉店 并排 在 一起 ， 五角星 螺丝 粉...",1
15,15,""" 第二次 来 吃 了 ， 因为 这里 价廉物美 ， 所以 老公 坚持 要 带 全家人 再来...",1
19,19,""" 位置 在工 农村 姚家 冒菜 旁边 的 旁边 那天 路 顺着 走 ' 都 在 一面 ' ...",0


In [99]:
print('Train Num: %d, Valid Num:%d\n\n'%(len(data_dict['price_level']['binary_train']),len(data_dict['price_level']['binary_valid'])))
for label in label_list:
    print(label)
    print('Train Num: %d, Valid Num: %d\n'%(len(data_dict[label]['sentiment_train']),len(data_dict[label]['sentiment_valid'])))
    

Train Num: 105000, Valid Num:15000


location_traffic_convenience
Train Num: 23618, Valid Num: 3243

location_distance_from_business_district
Train Num: 21320, Valid Num: 2968

location_easy_to_find
Train Num: 24395, Valid Num: 3484

service_wait_time
Train Num: 12237, Valid Num: 1763

service_waiters_attitude
Train Num: 62590, Valid Num: 9008

service_parking_convenience
Train Num: 6724, Valid Num: 954

service_serving_speed
Train Num: 16300, Valid Num: 2323

price_level
Train Num: 52180, Valid Num: 7503

price_cost_effective
Train Num: 24758, Valid Num: 3572

price_discount
Train Num: 40757, Valid Num: 5738

environment_decoration
Train Num: 51084, Valid Num: 7199

environment_noise
Train Num: 31555, Valid Num: 4479

environment_space
Train Num: 39602, Valid Num: 5509

environment_cleaness
Train Num: 38402, Valid Num: 5404

dish_portion
Train Num: 48083, Valid Num: 6888

dish_taste
Train Num: 99930, Valid Num: 14244

dish_look
Train Num: 29025, Valid Num: 4242

dish_recommendation
Tr

In [100]:
print(label_list)

['location_traffic_convenience', 'location_distance_from_business_district', 'location_easy_to_find', 'service_wait_time', 'service_waiters_attitude', 'service_parking_convenience', 'service_serving_speed', 'price_level', 'price_cost_effective', 'price_discount', 'environment_decoration', 'environment_noise', 'environment_space', 'environment_cleaness', 'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation', 'others_overall_experience', 'others_willing_to_consume_again']


In [101]:
# save
import os
if not os.path.exists('./process_data'):
    os.makedirs('./process_data')
for label in label_list:
    path = './process_data/'+label+'/'
    if not os.path.exists(path):
        os.makedirs(path)
    data_dict[label]['binary_train'].to_csv(path+label+'_binary_train.csv', index=False)
    data_dict[label]['binary_valid'].to_csv(path+label+'_binary_valid.csv', index=False)
    data_dict[label]['sentiment_train'].to_csv(path+label+'_sentiment_train.csv', index=False)
    data_dict[label]['sentiment_valid'].to_csv(path+label+'_sentiment_valid.csv', index=False)
    data_dict[label]['true_valid'].to_csv(path+label+'_true_valid.csv', index=False)

In [103]:
train.to_csv('data/train_tag.csv', index=False)
valid.to_csv('data/valid_tag.csv', index=False)
test.to_csv('data/test_tag.csv', index=False)